In [3]:
import gdal, osr
import numpy as np
from skimage.graph import route_through_array
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import os
import math
from osgeo import ogr
import fiona

In [4]:
def raster2array(rasterfn):
    #print('converting raster to array...')
    raster = gdal.Open(rasterfn)
    band = raster.GetRasterBand(1)
    array = band.ReadAsArray()
    return array

def array2raster(array, rasProp,newRasterfn):
    print('converting array to raster...')
    cols = array.shape[1]
    rows = array.shape[0]
    driver = gdal.GetDriverByName('GTiff')
    outRaster = driver.Create( newRasterfn, 
                              cols, rows,
                              bands=1, 
                              eType= gdal.GDT_Float32)
    outRaster.SetGeoTransform((rasProp.originX, 
                               rasProp.pixelWidth, 
                               0, rasProp.originY, 
                               0, rasProp.pixelHeight))
    
    outband = outRaster.GetRasterBand(1)
    outband.WriteArray(array)
    outRasterSRS = osr.SpatialReference()
    outRasterSRS.ImportFromWkt(rasProp.projRef)
    outRaster.SetProjection(outRasterSRS.ExportToWkt())
    outband.FlushCache()
    
class RasterProp:
    def __init__(self,
                 rasterFile,
                 sliceClass=None, slicing = False):
        self.raster = gdal.Open(rasterFile)
        self.geotransform = self.raster.GetGeoTransform()
        self.projRef      = self.raster.GetProjectionRef()
        self.originX = self.geotransform[0]
        self.originY = self.geotransform[3] 
        self.pixelWidth = self.geotransform[1] 
        self.pixelHeight = self.geotransform[5]
        
        if slicing:
            print('recomputing origin')
            x_ori_rel , y_ori_rel, xlen, ylen = sliceClass.relevantArea()
            self.originX, self.originY = pixel2coord(self.geotransform, 
                                                     x_ori_rel, 
                                                     y_ori_rel)
        
def coord2pixelOffset(rasProp,x,y):
    print('coordinate to pixel offsetting...')
    originX = rasProp.originX
    originY = rasProp.originY
    
    pixelWidth = rasProp.pixelWidth 
    pixelHeight = rasProp.pixelHeight

    xOffset = int((x - originX)/pixelWidth)
    yOffset = int((y - originY)/pixelHeight)
    return xOffset,yOffset


def pixel2coord(geoTrans, x, y):
    xoff, a, b, yoff, d, e = geoTrans

    xp = a * x + b * y + a * 0.5 + b * 0.5 + xoff
    yp = d * x + e * y + d * 0.5 + e * 0.5 + yoff
    return(int(xp), int(yp))

In [5]:
def createTotalCostRaster(factorPathList, 
                          weightList, 
                          rasProp,
                          rasterName, 
                          slicing=False, 
                          strPoint=None, 
                          endPoint=None):
    
    if not slicing:
        #print(factorNames[0])
        costArray = maxMinScale(raster2array(factorPathList[0]))*weightList[0]
        costArray[np.isnan(costArray)]=0
        
        for fpos in range(1,len(factorPathList)):
            #print(factorNames[fpos])
            factorArray = maxMinScale(raster2array(factorPathList[fpos]))*weightList[fpos]
            factorArray[np.isnan(factorArray)]=0
            #plt.imshow(factorArray)
            costArray = np.add(costArray, factorArray)
        costArray[np.isnan(costArray)]=0
        plt.imshow(costArray)
        plt.colorbar()
        array2raster(costArray, rasProp, rasterName)
        return costArray, rasProp
    
    
    else:
        sliceObj = Slicing(rasProp, strPoint, endPoint)
        raster = gdal.Open(factorPathList[0])
        band   = raster.GetRasterBand(1)
        
        x_ori_rel , y_ori_rel, xlen, ylen = sliceObj.relevantArea()
        
        sliceRasProp = RasterProp(factorPathList[0],
                                  slicing   = True, 
                                  sliceClass= sliceObj)
        
        array = band.ReadAsArray(xoff=x_ori_rel, 
                          yoff=y_ori_rel,
                          win_xsize=xlen,
                          win_ysize=ylen
                         )
        
        costArray = maxMinScale(array)*weightList[0]
        
        for fpos in range(1, len(factorPathList)):
            raster = gdal.Open(factorPathList[fpos])
            band   = raster.GetRasterBand(1)
            factorArray = maxMinScale(band.ReadAsArray(xoff=x_ori_rel, 
                          yoff=y_ori_rel,
                          win_xsize=xlen,
                          win_ysize=ylen
                         ))*weightList[fpos]
            
            costArray = np.add(costArray, factorArray)
        
        np.place(costArray, costArray==nan,0)
        array2raster(costArray, sliceRasProp, rasterName)
        plt.imshow(costArray)
        return costArray, sliceRasProp
    
    
def maxMinScale(array):
    return (array/abs(array.max()-array.min()))

In [6]:
def createPath(rasProp, costSurfaceArray,
               startCoord,stopCoord):
    '''returns an array of the same shape as costSurfaceArray with
    1 for path and 0 for other cells'''
    print('creating path...')
    # coordinates to array index
    startCoordX = startCoord[0]
    startCoordY = startCoord[1]
    startIndexX,startIndexY = coord2pixelOffset(rasProp,
                                                startCoordX,
                                                startCoordY)

    stopCoordX = stopCoord[0]
    stopCoordY = stopCoord[1]
    stopIndexX,stopIndexY = coord2pixelOffset(rasProp,
                                              stopCoordX,stopCoordY)
    # create path
    indices, weight = route_through_array(costSurfaceArray, 
                                          (startIndexY,startIndexX), 
                                          (stopIndexY,stopIndexX),
                                          geometric=True,
                                          fully_connected=True)
    
    
    indices = np.array(indices).T
    path = np.zeros_like(costSurfaceArray)
    path[indices[0], indices[1]] = 1
    print('path created...')
    return path

In [7]:
def getStartEndCord(file):
    '''For reading 'start' and 'end' coordindates from shape files - 
    used specifically for DC connection files'''
    
    shape = fiona.open(file)
    first = shape.next()
    strX, strY =first.get('properties').get('CoordX'), first.get('properties').get('CoordY')
    second = shape.next()
    endX, endY =second.get('properties').get('CoordX'), second.get('properties').get('CoordY')
    #return first
    return ((strX,strY) ,(endX,endY))


def writePath(costArray, dc, pathName):
    '''Calculating and writing path for DC Connections'''
    path = createPath(RasterProp(ecoFacRaster), costArray, 
                       getStartEndCord(dc)[0],  getStartEndCord(dc)[1])
    array2raster(path, RasterProp(ecoFacRaster), pathName)
    

## Evaluation Metrics

Upon calculation of path the following information is saved:

1. Length of path
2. People Affected
3. 'Similarness' to reference path (all equally weighted)
4. Land Quality
    4.1 Aggriculture
    4.2 Forest
    4.3 HVN
    4.4 Man-Made
    4.5 WasteLand
6. 'Cost' based on raster
    6.1 Eco
    6.2 Env
    6.3 Pub
    6.4 Inf
    6.5 All


In [8]:
ecoPath = os.path.abspath('01_Data500/fac_eco_onlySlope.tif')
envPath = os.path.abspath('01_Data500/fac_env.tif')
pubPath = os.path.abspath('01_Data500/fac_pub.tif')
infPath = os.path.abspath('01_Data500/fac_inf.tif')
citPath = os.path.abspath('01_Data500/city.tif')

In [9]:
ecoFac = raster2array(ecoPath)
envFac = raster2array(envPath)
pubFac = raster2array(pubPath)
infFac = raster2array(infPath)
citAre = raster2array(citPath)

In [10]:
def createPath(rasProp, costSurfaceArray,
               startCoord,stopCoord):
    '''returns an array of the same shape as costSurfaceArray with
    1 for path and 0 for other cells'''
    print('creating path...')
    # coordinates to array index
    startCoordX = startCoord[0]
    startCoordY = startCoord[1]
    startIndexX,startIndexY = coord2pixelOffset(rasProp,
                                                startCoordX,
                                                startCoordY)

    stopCoordX = stopCoord[0]
    stopCoordY = stopCoord[1]
    stopIndexX,stopIndexY = coord2pixelOffset(rasProp,
                                              stopCoordX,stopCoordY)
    # create path
    indices, weight = route_through_array(costSurfaceArray, 
                                          (startIndexY,startIndexX), 
                                          (stopIndexY,stopIndexX),
                                          geometric=True,
                                          fully_connected=True)
    
    
    indices = np.array(indices).T
    path = np.zeros_like(costSurfaceArray)
    path[indices[0], indices[1]] = 1
    print('path created...')
    return path

In [11]:
dcProjects = os.path.abspath('02_DC_Projects_DE//')
dc5Path = str(dcProjects+'\\DC_5.shp')

In [ ]:
%%time
allPaths = []
for eco in range(0,11,2):
    for env in range(0,11,2):
        for inf in range(0,11,2):
            for pub in range(0,11,2):
                if (eco==env==inf==pub==0):
                    continue;
                c_eco = eco/(eco+env+inf+pub)
                c_env = env/(eco+env+inf+pub)
                c_inf = inf/(eco+env+inf+pub)
                c_pub = pub/(eco+env+inf+pub)
                print([eco,env,inf,pub])
                totalCost = c_eco*ecoFac + c_env*envFac + c_pub*pubFac + c_inf*infFac + citAre
                path = createPath(RasterProp(ecoPath), totalCost,
                                  getStartEndCord(dc5Path)[0], getStartEndCord(dc5Path)[1])
                pathidx =np.nonzero(path)
                fileName = str(eco)+str(env)+str(inf)+str(pub)
                comName = os.path.abspath('02_DC_Projects_DE/02_dc5_paths/'+fileName+'.npy')
                np.save(comName,pathidx)

[0, 0, 0, 2]


C:\Users\Kumar\Anaconda3\envs\gis\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Collection.__next__() is buggy and will be removed in Fiona 2.0. Switch to `next(iter(collection))`.
  
C:\Users\Kumar\Anaconda3\envs\gis\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Collection.__next__() is buggy and will be removed in Fiona 2.0. Switch to `next(iter(collection))`.
  


creating path...
coordinate to pixel offsetting...
coordinate to pixel offsetting...
path created...
[0, 0, 0, 4]
creating path...
coordinate to pixel offsetting...
coordinate to pixel offsetting...
path created...
[0, 0, 0, 6]
creating path...
coordinate to pixel offsetting...
coordinate to pixel offsetting...
path created...
[0, 0, 0, 8]
creating path...
coordinate to pixel offsetting...
coordinate to pixel offsetting...
path created...
[0, 0, 0, 10]
creating path...
coordinate to pixel offsetting...
coordinate to pixel offsetting...
path created...
[0, 0, 2, 0]
creating path...
coordinate to pixel offsetting...
coordinate to pixel offsetting...
path created...
[0, 0, 2, 2]
creating path...
coordinate to pixel offsetting...
coordinate to pixel offsetting...
path created...
[0, 0, 2, 4]
creating path...
coordinate to pixel offsetting...
coordinate to pixel offsetting...
path created...
[0, 0, 2, 6]
creating path...
coordinate to pixel offsetting...
coordinate to pixel offsetting...
pa